In [1]:
 !pip install transformers

In [2]:
import os
import pandas as pd
import numpy as np

In [3]:
pip install openpyxl

     |████████████████████████████████| 242 kB 895 kB/s            
Note: you may need to restart the kernel to use updated packages.


In [4]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/6-class-bert-preprocessed-dataset/Bert_Preprocessed_Dataset.xlsx


In [5]:
df=pd.read_excel('/kaggle/input/6-class-bert-preprocessed-dataset/Bert_Preprocessed_Dataset.xlsx')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6405 entries, 0 to 6404
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6405 non-null   int64 
 1   ID          6404 non-null   object
 2   Text        6404 non-null   object
 3   Sentiment   6405 non-null   object
 4   Label       6405 non-null   int64 
 5   Token       6405 non-null   object
 6   Token_ID    6405 non-null   object
dtypes: int64(2), object(5)
memory usage: 350.4+ KB


In [6]:
pip install ktrain

     |████████████████████████████████| 25.3 MB 798 kB/s            
  Preparing metadata (setup.py) ... - done
     |████████████████████████████████| 22.3 MB 44.6 MB/s            
     |████████████████████████████████| 981 kB 49.8 MB/s            
  Preparing metadata (setup.py) ... - done
     |████████████████████████████████| 263 kB 58.4 MB/s            
     |████████████████████████████████| 2.8 MB 56.0 MB/s            
  Preparing metadata (setup.py) ... - done
     |████████████████████████████████| 468 kB 52.3 MB/s            
     |████████████████████████████████| 3.3 MB 53.7 MB/s            
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Created wheel for ktrain: filename=ktrain-0.30.0-py3-none-any.wh

In [7]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

In [8]:
import ktrain
from ktrain import text

In [9]:
train=df[:int(.8*len(df))]
test=df[int(.8*len(df)):int(.9*len(df))]
validation=df[int(.9*len(df)):]
x_train=train['Token_ID'].tolist()
y_train=train['Label'].tolist()
x_validation=validation['Token_ID'].tolist()
y_validation=validation['Label'].tolist()
x_test=test['Token_ID'].tolist()
y_test=test['Label'].tolist()

In [10]:
x_train= np.array(x_train)
x_validation= np.array(x_validation)
x_test= np.array(x_test)


In [11]:
categories = ['Angry','disgust','fear','happy','sad','surprise']

In [12]:
model_name = 'bert-base-multilingual-cased'
trans = text.Transformer(model_name,maxlen=50,class_names=categories)

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [13]:
train = trans.preprocess_train(x_train,y_train)
valid = trans.preprocess_test(x_validation,y_validation)

preprocessing train...


/opt/conda/lib/python3.7/site-packages/ktrain/text/textutils.py:246: UserWarning: Defaulting to English for language detection: could not detect language from documents. This may be due to empty or invalid texts being provided to detect_lang.
  warnings.warn('Defaulting to English for language detection: could not detect language from documents. '+\


language: en
train sequence lengths:
	mean : 26
	95percentile : 74
	99percentile : 173


Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 26
	95percentile : 76
	99percentile : 165


In [14]:
model = trans.get_classifier()

Downloading:   0%|          | 0.00/1.08G [00:00<?, ?B/s]

In [15]:
learner = ktrain.get_learner(model, train_data=train,val_data=valid, batch_size=12)

In [16]:
from sklearn.utils import class_weight
class_weight=class_weight.compute_class_weight(class_weight='balanced',classes=np.unique(y_train),y=y_train)
weight={i:class_weight[i] for i in range(6)}

In [17]:
learner.fit_onecycle(2e-5,20,class_weight=weight) 



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/20
427/427 [==============================] - 82s 145ms/step - loss: 1.7872 - accuracy: 0.1563 - val_loss: 1.7459 - val_accuracy: 0.2777
Epoch 2/20
427/427 [==============================] - 58s 137ms/step - loss: 1.7707 - accuracy: 0.2051 - val_loss: 1.8030 - val_accuracy: 0.1326
Epoch 3/20
427/427 [==============================] - 59s 138ms/step - loss: 1.7531 - accuracy: 0.2135 - val_loss: 1.7589 - val_accuracy: 0.1669
Epoch 4/20
427/427 [==============================] - 59s 138ms/step - loss: 1.7185 - accuracy: 0.2527 - val_loss: 1.6538 - val_accuracy: 0.2512
Epoch 5/20
427/427 [==============================] - 59s 139ms/step - loss: 1.6797 - accuracy: 0.3080 - val_loss: 1.5989 - val_accuracy: 0.3526
Epoch 6/20
427/427 [==============================] - 59s 139ms/step - loss: 1.6462 - accuracy: 0.3263 - val_loss: 1.6527 - val_accuracy: 0.3385
Epoch 7/20
427/427 [==============================] - 58s 137ms/ste

In [18]:
learner.validate(class_names=categories)

              precision    recall  f1-score   support

       Angry       0.48      0.55      0.51       137
     disgust       0.22      0.22      0.22        72
        fear       0.12      0.13      0.13        30
       happy       0.70      0.59      0.64       195
         sad       0.35      0.40      0.37       132
    surprise       0.35      0.28      0.31        75

    accuracy                           0.44       641
   macro avg       0.37      0.36      0.36       641
weighted avg       0.46      0.44      0.45       641



array([[ 75,  17,   5,   8,  21,  11],
       [ 18,  16,   3,   6,  24,   5],
       [  7,   3,   4,   5,   7,   4],
       [ 20,  11,   5, 116,  33,  10],
       [ 18,  18,  12,  22,  53,   9],
       [ 19,   8,   4,   8,  15,  21]])